# Lasso Scenario Creation Quickstart

In this notebook we will run through:

1. Using a configuration file to run lasso  
2. Setting up a base scenario and applying projects  
3. Transforming the standard network format to the MetCouncil expected format    
4. Exporting the network to a shapefile and csvs  

In [16]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.DEBUG)

# Remote I/O

In [11]:
input_dir = "../../travel-model-two-networks/data/interim/step8_standard_format"
output_dir = input_dir

In [18]:
parameters = Parameters()

2020-10-08 14:27:33, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


# Read Roadway and Transit Networks

In [13]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_file = link_file, 
    node_file = node_file, 
    shape_file = shape_file
)

2020-10-08 11:52:03, INFO: Reading from following files:
-Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\link.json
-Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\node.geojson
-Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\shape.geojson.
2020-10-08 11:56:12, INFO: Read 1634769 links from Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\link.json
2020-10-08 11:56:12, INFO: Read 644475 nodes from Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\node.geojson
2020-10-08 11:56:12, INFO: Read 869970 shapes from Z:/Data/Users/Sijia/MTC/github/travel-model-two-networks/data/interim/step8_standard_format\shape.geojson


In [14]:
roadway_net.links_df.columns

Index(['access', 'area', 'bike_access', 'bridge', 'drive_access',
       'fromIntersectionId', 'id', 'lanes', 'maxspeed', 'name', 'oneWay',
       'ref', 'roadway', 'shstGeometryId', 'shstReferenceId',
       'toIntersectionId', 'tunnel', 'u', 'v', 'walk_access', 'wayId', 'width',
       'county', 'length', 'model_link_id', 'A', 'B', 'rail_traveltime',
       'rail_only', 'locationReferences', 'shape_id', 'geometry'],
      dtype='object')

# Make travel model network

In [21]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = roadway_net, 
    parameters = parameters)

2020-10-08 14:57:04, INFO: Lasso base directory set as: z:\Data\Users\Sijia\MTC\github\Lasso


# Calculate number of lanes

In [61]:
m_net.determine_number_of_lanes(network_variable = 'numlanes')
m_net.links_df.numlanes.value_counts()

2020-10-08 21:52:04, INFO: Determining number of lanes
2020-10-08 21:57:33, INFO: Finished determining number of lanes using variable: numlanes


1    1486397
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: numlanes, dtype: int64

In [62]:
pd.crosstab(m_net.links_df.roadway, m_net.links_df.numlanes)

numlanes,1,2,3,4,5,6,7,8,9
roadway,,,,,,,,,
cycleway,99067,36,4,0,0,0,0,0,0
footway,247365,87,27,8,2,0,0,0,0
motorway,115,576,640,1026,314,96,6,1,0
motorway_link,3561,1184,187,44,7,0,0,0,0
primary,2555,11633,7466,3151,1367,235,71,14,0
primary_link,1356,110,35,3,2,0,0,0,0
residential,503743,33033,1424,369,90,7,0,0,0
secondary,25600,25120,7300,6494,3534,435,44,2,0
secondary_link,1357,188,18,2,1,0,0,0,0


# Calculate facility type

In [76]:
type(m_net.links_df.oneWay.iloc[0]) == str

True

In [79]:
"False" in "False"

True

In [78]:
[False, False]
','.join(map(str, [False, False]))

'False,False'

In [77]:
m_net.links_df.oneWay.value_counts()

False                                                                                                                             1463253
True                                                                                                                                88285
[False, False]                                                                                                                      54319
[True, True]                                                                                                                        10940
[False, False, False]                                                                                                               10228
[True, True, True]                                                                                                                   2909
[False, False, False, False]                                                                                                         1002
[True, True, True, True]          

In [72]:
set(m_net.links_df.oneWay.iloc[0])

{'T', 'e', 'r', 'u'}

In [71]:
m_net.links_df.oneWay.head(100)

0      True
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: oneWay, Length: 100, dtype: object